In [1]:
import pandas as pd
sys_df = pd.read_csv("us-covid-19-share-people-vaccinated.csv")
unwanted_regions = ["American Samoa", "Federated States of Micronesia"]
new_df = sys_df[sys_df['Entity'].isin(unwanted_regions) == False]

national_percentage = new_df.groupby('Day').mean()

new_df.head()

# April 8th = April 7th cumulative + average over 2 weeks of daily increase
# CI with smallest and largest increment

case_df = pd.read_csv("United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")
case_df.loc[case_df['state'] == 'MT']

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
5557,06/06/2020,MT,540,540.0,0.0,0,0.0,18,18.0,0.0,0,0.0,06/07/2020 02:23:35 PM,Agree,Agree
5638,09/14/2020,MT,9107,9107.0,0.0,86,0.0,138,138.0,0.0,3,0.0,09/15/2020 02:03:59 PM,Agree,Agree
5650,03/21/2021,MT,103060,96083.0,6977.0,94,8.0,1414,1414.0,0.0,0,0.0,03/22/2021 01:57:09 PM,Agree,Agree
5662,06/15/2020,MT,612,612.0,0.0,3,0.0,19,19.0,0.0,0,0.0,06/16/2020 02:10:11 PM,Agree,Agree
5802,02/01/2020,MT,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20897,03/08/2021,MT,100959,95421.0,5538.0,45,14.0,1381,1381.0,0.0,0,0.0,03/09/2021 02:47:41 PM,Agree,Agree
20933,08/27/2020,MT,6929,6929.0,0.0,144,0.0,98,98.0,0.0,0,0.0,08/28/2020 01:43:49 PM,Agree,Agree
21013,02/22/2021,MT,98810,98810.0,0.0,31,0.0,1341,1341.0,0.0,-1,0.0,02/23/2021 02:51:13 PM,Agree,Agree
21028,01/02/2021,MT,81944,81944.0,0.0,389,0.0,971,971.0,0.0,10,0.0,01/03/2021 02:53:25 PM,Agree,Agree
